Test authentication from DuckDB to Azure Storage Containers using user-delegated SAS tokens and custom token-credential chains

In [4]:
import duckdb
import utils
from azure.identity import ChainedTokenCredential, ManagedIdentityCredential, InteractiveBrowserCredential

In [5]:
tenant_id='3a2d9a5b-25c0-47da-9dff-842d10755773'
account_name = "azduckdb"
container_name = "data"
blob_path = "HuggingFaceFW/fineweb/CC-MAIN-2013-20/*.parquet"

In [7]:
credential = ChainedTokenCredential(ManagedIdentityCredential(), InteractiveBrowserCredential(tenant_id=tenant_id))

In [9]:
duckdb.sql(f"""
    CREATE OR REPLACE SECRET {account_name} (
        TYPE AZURE,
        CONNECTION_STRING '{utils.get_connection_string(credential, account_name, container_name)}',
        SCOPE 'az://{account_name}.blob.core.windows.net/{container_name}/'
    )
    """)

┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ true    │
└─────────┘

In [11]:
duckdb.sql("FROM duckdb_secrets()")

┌──────────┬─────────┬──────────┬────────────┬─────────┬──────────────────────┬────────────────────────────────────────┐
│   name   │  type   │ provider │ persistent │ storage │        scope         │             secret_string              │
│ varchar  │ varchar │ varchar  │  boolean   │ varchar │      varchar[]       │                varchar                 │
├──────────┼─────────┼──────────┼────────────┼─────────┼──────────────────────┼────────────────────────────────────────┤
│ azduckdb │ azure   │ config   │ false      │ memory  │ [az://azduckdb.blo…  │ name=azduckdb;type=azure;provider=co…  │
└──────────┴─────────┴──────────┴────────────┴─────────┴──────────────────────┴────────────────────────────────────────┘

In [10]:
duckdb.sql(f"SELECT COUNT(*) FROM 'az://{account_name}.blob.core.windows.net/{container_name}/{blob_path}'")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│      5433000 │
└──────────────┘

In [13]:
duckdb.sql(f"DESCRIBE TABLE 'az://{account_name}.blob.core.windows.net/{container_name}/{blob_path}'")

┌────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│  column_name   │ column_type │  null   │   key   │ default │  extra  │
│    varchar     │   varchar   │ varchar │ varchar │ varchar │ varchar │
├────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ text           │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ id             │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ dump           │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ url            │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ date           │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ file_path      │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ language       │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ language_score │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ token_count    │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
└────────────────┴─────────────┴─────────┴─────────